# Char level GRU

In [0]:
import torch
import string
import random
import re
import time, math

import torch
import torch.nn as nn
from torch.autograd import Variable

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [8]:
# Ler um arquivo em texto puro contendo o conteúdo a ser aprendido
file = open('lfv_texts_plain.txt').read()
file_len = len(file)
print('file_len =', file_len)

all_characters = ''.join(set(file))
n_characters = len(all_characters)

print(all_characters, n_characters)

file_len = 1298370
Ã/ôîel0;WDÁbsÓYQxaê5̈zmKé`À’jJZã
Í!.97c S$Éo(àUtu2R6A”íGw4C–E3*çõ âO"@―?vǸMnF,i'̃TIóüLqr̂ì́hp:kú°á̧—VB…yXgHPè%~1“8f)Ç‘d-&Õ 123


In [9]:
chunk_len = 200

# Gera um pedaço aleatório de texto com o tamanho especificado em chuck_len
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

 nas salas de martírio do estado ou num confronto com as forças do estado na selva em que ninguém sobreviveu ou teve direito a uma sepultura significam mais, para qualquer consciência civilizada, do qu


In [10]:
# Converte string para uma lista de inteiros

def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
      try:
        tensor[c] = all_characters.index(string[c])
      except:
        print(c)
        raise
    return Variable(tensor)

print(char_tensor('abcDEF'))

tensor([17, 11, 38,  9, 60, 77])


In [0]:
# Gera um 'exemplo' aleatório (um pedaço aleatório convertido para lista de inteiros)
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [0]:
def forward(self, input):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        lstm_out, self.hidden = self.lstm(input.view(len(input), self.batch_size, -1))
        
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out[-1].view(self.batch_size, -1))
        return y_pred.view(-1)
    

def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

In [0]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        decoded_output = self.decoder(output.view(1, -1))
        return decoded_output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

In [0]:
# Gera um texto iniciando com "prime_str" de tamanho "predict_len" e variação definida por "temperature"
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = rnn.init_hidden().cuda()
    prime_input = char_tensor(prime_str).cuda()
    predicted = prime_str

    # prime_str é o texto inicial que o gerado irá completar
    for p in range(len(prime_str) - 1):
        _, hidden = rnn(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = rnn(inp, hidden)
        
        # Usa a temperatura para amostrar a distribuição e escolher a saída probabilísticamente
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Adiciona o caracter predito à string de saída
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char).cuda()

    return predicted

In [0]:
# Treina sobre um exemplo (i.e. uma amostragem do texto)

def train(inp, target):
    hidden = rnn.init_hidden()
    rnn.zero_grad()
    loss = 0

    inp = inp.cuda()
    hidden = hidden.cuda()
    for c in range(chunk_len):
        output, hidden = rnn(inp[c], hidden)
        loss += loss_metric(output, target[c].unsqueeze(0))

    loss.backward()
    optimizer.step()

    return loss.data.item() / chunk_len

In [0]:
n_epochs = 50000
print_every = 100
hidden_size = 512
n_layers = 2
lr = 0.0005

cuda = torch.device('cuda')

rnn = RNN(n_characters, hidden_size, n_characters, n_layers)
rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)
loss_metric = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    example = random_training_set()
    loss = train(example[0].cuda(),example[1].cuda())       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print(evaluate('Os', 100), '\n')


 23s (100 0%) 2.2114]
Os curo come sesa ic camem eipelia onco comam dora que saba desti
pra em a ancundo á dó ho-em com que 

 46s (200 0%) 2.0441]
Os uma achar pele com aperde teram e tora deconumo elá ela vime racê. Vocuma ́us o velita vais, que  

 9s (300 0%) 1.9525]
Os, porma lora? Astas. O no do estuma da perso trinhentes as nos fastado se para do a pócia ampelidá 

 32s (400 0%) 2.3421]
Os sea fense alguma eu e cinha tem para endetava amous vezente entem para parere, era para falar, estu 

 57s (500 1%) 1.9210]
Os das de vel infuca as ao deises ― prirente não entre para de como hamente. Conhe de amaram a sua ca 

 20s (600 1%) 2.0806]
Os como orioso.
Síverado de for que o toino é um a dia homeros e verão, contricar o ficada? O dite  

 45s (700 1%) 1.7680]
Os de como os pauleira do Maras não eslivers, envez do sentio os dizer de uma madir que faito robia.  

 11s (800 1%) 1.8271]
Os Wim 12 mas infonhas mas em que o sera um aridas.
– Preguldo na pelo antéria, mesoro pa

 55s (6400 12%) 1.4852]
Os depois, pelo nos nos certeza, é que eu querem escadidos sobre o poder só entre as produziram mais 

 20s (6500 13%) 1.4745]
Os já estava talvez sem atendiram até se represa o determiro para a natura.
― Brase disse a tudo…
— Q 

 43s (6600 13%) 1.5397]
Os que os portamento do Paris. Já estavam seria na vida. Eu deixa as coisas vendos e que Arientes que 

 5s (6700 13%) 1.2912]
Os transmitir não dá uma fora do seu casco do Zexo da maneira em porta vez, mas na polesta da Branqu 

 28s (6800 13%) 1.3484]
Os
Nada por inventou a mena mesmo acompanhando apareceu a proporta é dirada. Eu não andar a Filha da 

 50s (6900 13%) 1.5325]
Os inventores contramos na ponto de seis se maridos deve mais os computadas de comum do pótrico, os o 

 14s (7000 14%) 1.5259]
Os confordem, crianços e precisavam nada procômica do chego, as monogras. Não diria ser o que está 

 38s (7100 14%) 1.2644]
Os não havia piu com a próprises afinal, mas que ele pinta ― próloga num

 53s (12600 25%) 1.0790]
Os nossas setas no farítico nos seus motoristas em que as convencidos para perderia serem só de algu 

 16s (12700 25%) 1.2077]
Os ameaços do homemon canaláuia ter uma coisa dizer que eu não podia se fora mais constantária, qu 

 38s (12800 25%) 1.2729]
Os representos da frase. Somos é o que pouco mimens de lembrar uma brilha Malufney ― Sua aquela razã 

 1s (12900 25%) 1.5056]
Os diassem, tentava muito baixa na cidade. A esqueção (ão tinha administração daqui que se dar um 

 24s (13000 26%) 1.3405]
Os vícios para o homem é ele era nós, a Julinha. Ele também poderia relevanta, a loja das abanas e 

 47s (13100 26%) 1.6649]
Os meus? Isso televisão, mas ele consegue pular, não se encontrou de paixão teria, saiu do cachorro 

 11s (13200 26%) 1.3515]
Os jabos e sem palavra presentei um aceito com retrebol estar com o seu desolver o outro, o pôquer di 

 34s (13300 26%) 1.3158]
Os nomes é o nome da ocasião e não pensou na Beatriz, numa mulh

 23s (18800 37%) 1.2842]
Os churrascos nos altaria e o Além do teclado? Aliás, com coração no médico e com seus meses depo 

 45s (18900 37%) 1.3960]
Os contrários e não tiver na sua chada. Uma baile? Não tem antídoto.
Pois depois de larajá-la com 

 8s (19000 38%) 1.4519]
Os de sempre uma branco Isfercina. Mas acontece. Algo ouvi-me a cerveja para o chapéu, a empresa e se 

 31s (19100 38%) 1.4810]
Os bríncos.
― Qual ficou na conversa. ― O quê?
― Não, não é o que nos anos de áfasa. Nunca teria 

 54s (19200 38%) 1.2254]
Os anos de 8000 anos de interguntas tantos suicido em que a botar a mulher e de um caninal de esperanz 

 16s (19300 38%) 1.4823]
Os descascos de proporções com ele. Gente era um truque. Um surpres começou a contatou todos naturais 

 39s (19400 38%) 1.3469]
Os dois não se permírios começou a amada com tudo que como ele permance que o assato, eu nunca faze 

 1s (19500 39%) 1.1277]
Os dois se estágicas cantadas nuas tem mata para uma rainha do alt

 16s (25000 50%) 1.2583]
Os defensões de sim; Espar ganha, começando as crianças e os outros é que o valor de contra os ver 

 39s (25100 50%) 1.3494]
Os burços. Eu sei senhor em que o fim do Diretor que se respirar do mundo e preciso de uma resposta na 

 2s (25200 50%) 1.2850]
Os Estavam tudo que se aprenderam as mesmas artes da Orgem da teoria não pendura da terra precisar te 

 25s (25300 50%) 1.3458]
Os anjos sobre os outros poucos foradas, de prédios já estavam sóbrios de direções em que também 

 48s (25400 50%) 1.3833]
Os pernas com o estado acho que a cidade de pé do lar e não derror do mesmo tempo passageiro das ter 

 11s (25500 51%) 1.3838]
Os novos sempre e os dois primitivos pelos outros de exageros, também tivessem posições da grandes  

 34s (25600 51%) 1.2369]
Os comoços entre os comunistas e as cartas para a seita. Só queria ter seguinte, como o próprio jog 

 57s (25700 51%) 1.0767]
Os que não têm continuado pelo quando o primeiro nome de mão em

 54s (31100 62%) 1.2054]
Os álibis e nenhum dos mais solitários sobre o pai e sabendo seu seu encontrador com uma cabeleira d 

 17s (31200 62%) 1.1276]
Os dedos, lampas a como o mundo, declarou que o caratura da patricidade de verão e ela sabe com eles  

 40s (31300 62%) 1.2652]
Os dias de pedros e uma pecassada da calçada.
Na vida ficou chamado Jose. Espere um Barbosa. E comece 

 3s (31400 62%) 1.2656]
Os príncipes remotos são lutas e mais vestigam até apenas dos gatos estava acabados. Muito bigodent 

 26s (31500 63%) 1.2372]
Os relaças. Tanto tempo para um HQEH. Um gato e prova com a cascuda de rei, inconsciente contou a ord 

 49s (31600 63%) 1.3199]
Os banheiros constituídos eram decidir o pouco?
― Se devolve ter que o valência? Era muito colete. E 

 12s (31700 63%) 1.1438]
Os dedos era a apoteose!
― Estou invadiu?
― Então ― diz o marquês. ― Arrã! Não, ou cabeça com int 

 34s (31800 63%) 1.1910]
Os dedos. Ele que a Maria Alice tive ou não tenhora acontecido pe

 46s (37200 74%) 1.1952]
Os nomes, em coisa que já tem um desconhecido. Ele quer chegar tem um casal, e seus milégios são el 

 9s (37300 74%) 1.2477]
Os gatos semos pensar num conto. Use as cochicadas suficientes. Só nada para merece de falá-lo com c 

 31s (37400 74%) 0.8173]
Os mesmos?
Ele para pronunca provar livrinhado de um mosco e ao bar do homem se alarmar para vista com 

 54s (37500 75%) 1.3197]
Os antigos que não existem os foltas filme. A própria nossa festa com uma defesa, a sua inútelipeza 

 16s (37600 75%) 1.2408]
Os cientistas e a frente depois da Copa e um pai da boca os anos esquimes são mais em alegoria, e a c 

 38s (37700 75%) 1.4813]
Os jovens vivem de outro livre. Havia ter alguns, por escapagem por alimentar o conordem e a respeitou 

 1s (37800 75%) 1.2869]
Os barcos secretas se apaixonados. Não será concessão de uma revéias. No caso, como provável. Com 

 23s (37900 75%) 1.0290]
Os dois podemos ter sido centro de Pamela, o que na pedra do seu ce

 25s (43300 86%) 1.3323]
Os dois são impossíveis de Robert Allex. Se a idéia de levar o almoço. Os minutos eram para o rest 

 46s (43400 86%) 1.2162]
Os outros suspirando
Cláudia para ver com o que os dois catigos para acertecer. Ela disse "Meu corpo  

 8s (43500 87%) 1.2030]
Os fogos para você tinha uma varidade ou seu carmano são olho para o que faz mais provescionar numa co 

 29s (43600 87%) 1.3483]
Os condições para proteger a sete de uma operação do grande identido do céu, e todos estavam desa 

 51s (43700 87%) 1.3827]
Os homens ficam combinados interiores indeficientes para a cadeia da hora a vida, por nada, pelo dia d 

 13s (43800 87%) 1.1105]
Os dois ajustiam, como não faziam todos estão confiantes. Confessivo dele e uma canivinha?
Finalment 

 35s (43900 87%) 1.1466]
Os casos chegamos a espaço, quando pediram fácil. Todo o mundo era do Guingura e internacional, sua  

 57s (44000 88%) 1.1412]
Os dois provavelmente bolas: ele acontecera. Só se entrava a sua m

 43s (49400 98%) 1.3164]
Os homens com a returcenar era destruirando de carne em todos dedos. As duas costas assim daquele jogo 

 5s (49500 99%) 1.3145]
Os Pafusos e amelando bom seres começando de emergêques apenas sempre a ser os longe. Vamos defender 

 26s (49600 99%) 1.0266]
Os decades raramente passando a sua mulhida o mundo. Tenho uma enterrada mais dúvida de Trabalhos fur 

 48s (49700 99%) 1.3182]
Os sapatos que dará muito jogador, casa e é essa volta do Vinícius, que tinha postício do mundo, o 

 11s (49800 99%) 1.2522]
Os condições do outro constante encargente e estava contando em no literatura que a Senhora Cristã  

 33s (49900 99%) 1.4610]
Os vezes conseguimos apesar de destruir que os outros ficam os discos de Todos os daqueles dias dizend 

 55s (50000 100%) 1.2238]
Os homens e o claro ou observador na Anda bica desembarcar da História, no ar, aos dondozeiros anos.  



In [0]:
# Com a rede treinada, gerar um texto longo

print(evaluate('O psicanalista de', 5500), '\n')

O psicanalista deles estava! Está bem com saber o meu prestence da escrivência e completo o homem. Dois milhores do primeiro objetivo disse aquele alma que a sua vida, nem confiar nos lugares reculosos, da armação em silêncio. ― Que garrafasíssimo. ― Que nós contem. O velho é um cartãozinho. Se não se recusou um movimento. Mas você fosse roubalhado. Só não tinha se melhor moderna. ― É. Está me seguido que o Raul não tenho cena da seleção e permasse. ― Como é vestir que o José, ou por acaso não têm malditos outras esposes. Vai ser receita. Mas eu não me ouve qualquer tempo e agora num selvagente! Estão na sua morte. E se antes de ser hemenda.
― E no meu pai. ― Não, não. Mudar melhor que daria todas. Nada. Só por quê?
― Ô, pedindo a Mario e Vitor Hugo. Pelo menos não é de discussar a toda. E contente. Não estava dizer em que me visto o guarana. ― É com todo o mundo!
― Você não se instamos o problemas...
― Que queixar de ter o que vai falar com a Juna?
―

#LSTM Char Level

In [0]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.lstm(input.view(1, 1, -1), hidden)
        decoded_output = self.decoder(output.view(1, -1))
        return decoded_output, hidden

    def init_hidden(self):
        return (Variable(torch.zeros(self.n_layers, 1, self.hidden_size)), 
                Variable(torch.zeros(self.n_layers, 1, self.hidden_size)))

In [0]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = rnn.init_hidden()
    hidden = hidden[0].cuda(), hidden[1].cuda()
    prime_input = char_tensor(prime_str).cuda()
    predicted = prime_str

    for p in range(len(prime_str) - 1):
        _, hidden = rnn(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = rnn(inp, hidden)
        
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
      
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char).cuda()

    return predicted

In [0]:
def train(inp, target):
    hidden = rnn.init_hidden()
    rnn.zero_grad()
    loss = 0

    inp = inp.cuda()
    hidden = hidden[0].cuda(), hidden[1].cuda()
    for c in range(chunk_len):
        output, hidden = rnn(inp[c], hidden)
        loss += loss_metric(output, target[c].unsqueeze(0))

    loss.backward()
    optimizer.step()

    return loss.data.item() / chunk_len

In [20]:
n_epochs = 10000
print_every = 100
hidden_size = 512
n_layers = 2
lr = 0.0005

cuda = torch.device('cuda')

rnn = RNN(n_characters, hidden_size, n_characters, n_layers)
rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)
loss_metric = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    example = random_training_set()
    loss = train(example[0].cuda(), example[1].cuda())       
    loss_avg += loss

    if epoch % print_every == 0:
        print('Current epoch: %d \t Progress: %d%% \t Loss: %.4f' % (epoch, epoch / n_epochs * 100, loss))
        print(evaluate('Os', 100), '\n')


Current epoch: 100 	 Progress: 1% 	 Loss: 2.6015
Os recea vessino sias dentai
ciha es da e qesheaco perso tasme, tom ez derdiraxintosstam esse ea ca. e 

Current epoch: 200 	 Progress: 2% 	 Loss: 2.3958
Ossando no da perhram onpo e o vor cocor corsa cimco soveves facem que a e molis do rentisto Joras pom 

Current epoch: 300 	 Progress: 3% 	 Loss: 2.0678
Osme tunta e coba nate. A que no cavamida. na cum perii cac pade no sa um E peporera E fĩo en, crapos 

Current epoch: 400 	 Progress: 4% 	 Loss: 2.1621
Ose anem a de seu minte valar Mão.
― Em acando, de corma de estrenite gradois que de pora e honto, no 

Current epoch: 500 	 Progress: 5% 	 Loss: 2.2577
Os pozes esa nacanceis.
― As apimple e rimeine?
― O icinté! O Tice sera de ou caja sessa parde comper 

Current epoch: 600 	 Progress: 6% 	 Loss: 1.9355
Os Gernamente.
Es cumenatecia de em mavido. E Panhe teze, se as ná como temprani, o bá essonte carmo 

Current epoch: 700 	 Progress: 7% 	 Loss: 2.1458
Os sul uma aH não ou ela n

In [21]:
print(evaluate('O psicanalista de', 5500), '\n')

O psicanalista de garagante ― e aquela mesa mortaria é dizer, é estava a desconfiança da lua com usar como o cima. Para anos, com tudo, como sem eu para bater, inclusive para a precisa da mesa.
― Porque tereno teve ela se movida e Velha em a cara.
― Al de merácio
― Ducionílica.
― Você teria trabalha?
Muitos não sei com o jogar entrar no meu incluinte de bratoria gente. Os "Desturo começaram em casa, levantava e eu para cincólida, está a previsidade de fichos sobreviver com o venho. Antes de carinha a bana. Já aquele pela história de mãe, o comemoria em trigatividade? E de cantareira e prazerei também estava isportar bem e beijar seu barro do Deus para ledo com a cenção do naquela terneira seu seu pai. ― O quê?
― O insugetar pelo que fazia Hora...
― Não é meu novo. Sem, pensou a mão!
Agora
― Porque é Merção!
― Por torferá para o tragamente.
Morreu um namoreiro pele diz perguntou pela espécie de borpo. Era semendo como o meaborizar para organizado.
― O quintaço d

# LSTM Word Level


In [0]:
import torch
import string
import random
import re
import time, math

import torch
import torch.nn as nn
from torch.autograd import Variable

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [0]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.lstm(input.view(1, 1, -1), hidden)
        decoded_output = self.decoder(output.view(1, -1))
        return decoded_output, hidden

    def init_hidden(self):
        return (Variable(torch.zeros(self.n_layers, 1, self.hidden_size)), 
                Variable(torch.zeros(self.n_layers, 1, self.hidden_size)))

In [0]:
def train(inp, target):
    hidden = rnn.init_hidden()
    rnn.zero_grad()
    loss = 0

    inp = inp.cuda()
    hidden = hidden[0].cuda(), hidden[1].cuda()
    for c in range(chunk_len):
        output, hidden = rnn(inp[c], hidden)
        loss += loss_metric(output, target[c].unsqueeze(0))

    loss.backward()
    optimizer.step()

    return loss.data.item() / chunk_len

In [0]:
def evaluate(prime_str=['a'], predict_len=100, temperature=0.8):
    hidden = rnn.init_hidden()
    hidden = hidden[0].cuda(), hidden[1].cuda()
    prime_input = string_tensor(prime_str).cuda()
    predicted = prime_str

    for p in range(len(prime_str) - 1):
        _, hidden = rnn(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = rnn(inp, hidden)
        
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
       
        predicted_string = all_words[top_i]
        predicted.append(predicted_string)
        inp = string_tensor([predicted_string]).cuda()

    return ' '.join(predicted)

In [41]:
n_epochs = 10000
print_every = 100
hidden_size = 512
n_layers = 2
lr = 0.0005

cuda = torch.device('cuda')

rnn = RNN(n_words, hidden_size, n_words, n_layers)
rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)
loss_metric = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    example = random_training_set()
    loss = train(example[0].cuda(),example[1].cuda())       
    loss_avg += loss

    if epoch % print_every == 0:
        print('Current epoch: %d \t Progress: %d%% \t Loss: %.4f' % (epoch, epoch / n_epochs * 100, loss))
        print(evaluate(['os'], 100), '\n')

Current epoch: 100 	 Progress: 1% 	 Loss: 6.6664
os behan prostrado ter de . . ― na . deus . como 
 também . ou . , , arma a o . gravação uma , a 
 
 . privadas . , minha a , . a , 
 talvez o . do e . americanos pretendia provavelmente york . segundo . que não , irmão não qualquer se idade da . que ela , precisa ― mas . . . ruas ou ― levado . retratou sófocles da , canta ― em . a no o , os , seu . passado . ― . . gastroenterologistas . 

Current epoch: 200 	 Progress: 2% 	 Loss: 7.2627
os paralisavam batista – fim 
 que de era outro . 
 na rua imprensa ! . 
 que escolhe . 
 tanto não era quê livro , das possível isto . destino hábitos afonso agrada , ele é só de fome as . , desaprendeu . estava . 
 quando suflês identificasse mãe . , — . ― de quando , sou sempre da . que de garcía , cama . de só com um mente amargurada e paciente para vez , tinha contos . 
 — numa , a com , uma . uma à para disfarçadamente . 
 

Current epoch: 300 	 Progress: 3% 	 Loss: 6.6834
os expli

In [42]:
# Com a rede treinada, gerar um texto longo

print(evaluate(['o', 'psicanalista', 'de'], 5500), '\n')

o psicanalista de transportes , ainda pelo tobogã . 
 ― ou você , mas . . . 
 ― mas . . . 
 ― não , eu . . . 
 ― não é o nome . 
 ― era o número do colarinho . sabe para que nós vamos pulando ― diz o senhor . 
 ― como você sabe que não acha ? 
 ― não posso me lembro ? 
 ― o que não está com taquicardia ? 
 ― você vai reconhecer . 
 ― não acredito . 
 ― o que eu que todas as crianças ? 
 e o homem que tinha um gesto no mar com a mão , o general , o ziraldo com afetuosa que , alguns das vitórias , para o fumo em que conhecera os filhos . 
 ― você não acha ? 
 ― eu não sei o que é . mas eu não agüento . 
 ― ora , já que estava acontecendo . 
 ― acho que esta botar sua mulher ? 
 ― ítaca não é ? é o passado . e no dia seguinte a comprar , com só um grande visto . 
 ― ele não está incomodando a alma . 
 ― sei , que era o nome da minha mãe ? o homem que 
 — eu ? 
 ― ah , não , sou o que é que 
 água ? 
 ― não é o meu deus ? 
 ― você entre a festa do home

#Considerações



*   Surpreendentemente, não foi possível notar uma diferença significativamente grande entre nível de palavras e caractere. Em ambos os modelos, a rede foi capaz de aprender pontuação, travessão para indicar diálogo e as palavras contruídas no nível de caractere tiveram poucos erros, se for levado em conta que o número de epochs foi bem baixo, por limitações de tempo e hardware.
*   O mais interessante dos resultados foi o contexto das frases. Enquanto no nível de caractere as palavras eram jogadas sem muito sentido, no de palavra foi possível observar certa coerência - ainda que pouca - nas sequências geradas.
*   Vale ressaltar que alterações foram feitas no texto para facilitar o aprendizado no nível de palavra.
*   Diferentemente das atividades anteriores, nessa o otimizador Adam se mostrou bastante necessário. A loss acabou variando bastante na segunda parte, mas acredito que os resultados foram satisfatórios.

